# We add stations in the database.

In [8]:
import sqlite3
import requests

# Connexion à la base de données
conn = sqlite3.connect('objets_trouves.db')
cursor = conn.cursor()

In [9]:
# Création de la table pour stocker les données de fréquentation des gares
cursor.execute("""
    CREATE TABLE IF NOT EXISTS gares (
        id INTEGER PRIMARY KEY,
        nom_des_gares TEXT NOT NULL,
        latitude FLOAT,
        longitude FLOAT,
        frequentation_2019 INTEGER,
        frequentation_2020 INTEGER,
        frequentation_2021 INTEGER,
        frequentation_2022 INTEGER
    )
""")

In [10]:
# Liste des gares avec leur latitude et longitude
latitudes_longitudes = {
    "Paris Gare de Lyon": (48.844321, 2.373773),
    "Paris Montparnasse": (48.840744, 2.320817),
    "Paris Gare du Nord": (48.880931, 2.355238),
    "Paris Saint-Lazare": (48.876986, 2.325173),
    "Paris Est": (48.876082, 2.358306),
    "Paris Austerlitz": (48.841935, 2.365084),
    "Paris Bercy": (48.839380, 2.386402)
}

In [11]:
# Liste des gares à récupérer
gares = ["Paris Gare de Lyon", "Paris Montparnasse", "Paris Gare du Nord", 
         "Paris Saint-Lazare", "Paris Est", "Paris Austerlitz", "Paris Bercy"]

In [12]:
# Récupération des données de fréquentation pour chaque gare
for gare in gares:
    url = f"https://ressources.data.sncf.com/api/records/1.0/search/?dataset=frequentation-gares&q={gare}&facet=nom_gare&facet=total_voyageurs&timezone=Europe%2FParis"
    response = requests.get(url)

    if response.ok:
        data = response.json()
        records = data['records']
    else:
        print(f'Erreur lors de la requête pour la gare {gare}:', response.status_code)
        continue

    # Stockage des données de fréquentation dans la base de données
    for record in records:
        nom_gare = record['fields']['nom_gare']
        if nom_gare != gare:
            continue
        latitude, longitude = latitudes_longitudes[nom_gare]
        frequentations = record['fields']
        frequentation_2019 = frequentations['total_voyageurs_2019']
        frequentation_2020 = frequentations['total_voyageurs_2020']
        frequentation_2021 = frequentations['total_voyageurs_2021']

        # TODO: Calculer la fréquentation pour 2022
        frequentation_2022 = frequentations['total_voyageurs_2021'] * 1.1
        cursor.execute("INSERT INTO gares (nom_des_gares, latitude, longitude, frequentation_2019, frequentation_2020, frequentation_2021, frequentation_2022) VALUES (?, ?, ?, ?, ?, ?, ?)", (nom_gare, latitude, longitude, frequentation_2019, frequentation_2020, frequentation_2021, frequentation_2022))

In [13]:
# Calcul de la fréquentation approximative pour 2022
frequentation_2022 = frequentations['total_voyageurs_2021'] * 1.1
cursor.execute("INSERT INTO gares (nom_des_gares, latitude, longitude, frequentation_2019, frequentation_2020, frequentation_2021, frequentation_2022) VALUES (?, ?, ?, ?, ?, ?, ?)", (nom_gare, latitude, longitude, frequentation_2019, frequentation_2020, frequentation_2021, frequentation_2022))

In [14]:
# Enregistrement des changements
conn.commit()
conn.close()